# Generate Graphs

This section is for generating random, connected graphs, with networkx. In order to do that, for sparse graph first generate a tree, then add the desired number of edges. For dense graphs, use the Erdős - Rényi graph generator.

In [2]:
import os
import random
import networkx as nx
import math

In [ ]:
"""Generate non scaled and scaled random graphs"""
GRAPH_NODES = [2_000, 4_000, 6_000, 8_000, 10_000, 20_000, 40_000, 60_000, 80_000, 100_000]
# GRAPH_NODES = [200_000, 400_000, 600_000, 800_000, 1_000_000]
# EDGE_RANGES = [2_000, 4_000, 6_000, 8_000, 10_000, 20_000, 40_000, 60_000, 80_000, 100_000, 200_000, 400_000, 600_000, 800_000, 1_000_000]
# graph edge numbers will be 10n

SAMPLE = 10  # Number of graphs with the same parameters

FOLDER = os.path.join(os.path.expanduser('~'), 'szakdolgozat', 'python', 'graphs', 'random_graphs_n32_2n')
SCALED_FOLDER = os.path.join(os.path.expanduser('~'), 'szakdolgozat', 'python', 'graphs', 'random_graphs_n32_2n_1000')

for n in GRAPH_NODES:
    m = n**1.5
    C = 2*n
    for i in range(SAMPLE):

        # Store the graph in 3 lists. Faster than networkx
        edges = []
        weights = []
        bounds = []

        scaled_weights = []

        # Generate the graph
        G = nx.gnm_random_graph(n, m)
        while not nx.is_connected(G):
            G = nx.gnm_random_graph(n, m)

        # Fill the lists, and generate edge weights
        for u in G.nodes():
            bounds.append(len(edges))
            for v in G.neighbors(u):
                edges.append(v)
                weights.append(random.choice(range(1, C+1)))
                scaled_weights.append(random.choice(range(1000, C+1, 1000)))

        # Write to file
        filename = f'{n}_{i}.txt'
        path = os.path.join(FOLDER, filename)
        with open(path, 'w') as f:
            for edge in edges:
                f.write(f'{edge} ')
            f.write('\n')
            for weight in weights:
                f.write(f'{weight} ')
            f.write('\n')
            for bound in bounds:
                f.write(f'{bound} ')

        filename = f'{n}_{i}.txt'
        path = os.path.join(SCALED_FOLDER, filename)
        with open(path, 'w') as f:
            for edge in edges:
                f.write(f'{edge} ')
            f.write('\n')
            for weight in scaled_weights:
                f.write(f'{weight} ')
            f.write('\n')
            for bound in bounds:
                f.write(f'{bound} ')


In [4]:
"""Generate scaled random graphs"""
GRAPH_NODES = [2_000, 4_000, 6_000, 8_000, 10_000, 20_000, 40_000, 60_000, 80_000, 100_000]
# GRAPH_NODES = [200_000, 400_000, 600_000, 800_000, 1_000_000]
# EDGE_RANGES = [2_000, 4_000, 6_000, 8_000, 10_000, 20_000, 40_000, 60_000, 80_000, 100_000, 200_000, 400_000, 600_000, 800_000, 1_000_000]
# graph edge numbers will be 10n


SAMPLE = 10  # Number of graphs with the same parameters

FOLDER = os.path.join(os.path.expanduser('~'), 'szakdolgozat', 'python', 'graphs', 'random_graphs_10n_2n_1000')
C = 1000
# Sparse Graphs
for n in GRAPH_NODES:
    m = 10*n
    for i in range(SAMPLE):

        # Store the graph in 3 lists. Faster than networkx
        edges = []
        weights = []
        bounds = []

        # Generate the graph
        G = nx.gnm_random_graph(n, m)
        while not nx.is_connected(G):
            G = nx.gnm_random_graph(n, m)

        # Fill the lists, and generate edge weights
        for u in G.nodes():
            bounds.append(len(edges))
            for v in G.neighbors(u):
                edges.append(v)
                weights.append(random.choice(range(1, 2*n+1))*100)
        
        # Write to file
        filename = f'{n}_10n_2n_1000_{i}.txt'
        path = os.path.join(FOLDER, filename)
        with open(path, 'a') as f:
            for edge in edges:
                f.write(f'{edge} ')
            f.write('\n')
            for weight in weights:
                f.write(f'{weight} ')
            f.write('\n')
            for bound in bounds:
                f.write(f'{bound} ')


In [19]:
def dijkstra_save_operations(source: int, heap, edge_list, edge_weight, node_bounds):
    """
        give back the lists of evaluated operations
    """
    operation_types = []
    operation_args_1 = []
    operation_args_2 = []

    visited_nodes = heap

    unvisited_nodes = [True]*(len(node_bounds)-1)
    unvisited_nodes[source] = False

    # Length of path from source to each node
    shortest_path = [0]*(len(node_bounds)-1)
    # Parent of each node on the shortest path
    parent_node = [-1]*(len(node_bounds)-1)
    parent_node[source] = 0

    # push source node
    operation_types.append('push')
    operation_args_1.append(source)
    operation_args_2.append(0)
    visited_nodes.push((source, 0))

    while len(visited_nodes) > 0:

        #pop
        operation_types.append('pop')
        operation_args_1.append(0)
        operation_args_2.append(0)
        v = visited_nodes.pop() # (name, key) tuple

        shortest_path[v[0]] = v[1]

        neighbors = zip(edge_list[ node_bounds[v[0]]: node_bounds[v[0]+1] ], edge_weight[ node_bounds[v[0]]: node_bounds[v[0]+1] ])
        
        for u_name, u_key in neighbors:
            if u_name in visited_nodes and v[1] + u_key < visited_nodes.getkey(u_name):

                #decrease_key
                operation_types.append('decrease_key')
                operation_args_1.append(u_name)
                operation_args_2.append(v[1] + u_key)
                visited_nodes.decrease_key(u_name, v[1] + u_key)

                parent_node[u_name] = v[0]

            elif unvisited_nodes[u_name]:
                
                # push
                operation_types.append('push')
                operation_args_1.append(u_name)
                operation_args_2.append(v[1] + u_key)
                visited_nodes.push((u_name, v[1] + u_key))
                
                parent_node[u_name] = v[0]
                unvisited_nodes[u_name] = False
    return operation_types, operation_args_1, operation_args_2

In [8]:
"""dijkstra save operations

"""
import os
from heaps.binary_heap import BinaryHeap
GRAPH_NODES = [2_000, 4_000, 6_000, 8_000, 10_000, 20_000, 40_000, 60_000, 80_000, 100_000]

SAMPLE = 10
SOURCE_FOLDER = os.path.join(os.path.expanduser('~'), 'szakdolgozat', 'python', 'graphs', 'random_graphs_n32_2n_100')
DESTINATION = os.path.join(os.path.expanduser('~'), 'szakdolgozat', 'python', 'graph_operations', 'random_n32_2n_100',)

for n in GRAPH_NODES:
    for i in range(SAMPLE):
        edge_list = []
        edge_weight = []
        node_bounds = []

        # read the graph
        filename = f'{n}_{i}.txt'
        path = os.path.join(SOURCE_FOLDER, filename)
        with open(path, 'r') as f:
            edge_list = [int(x) for x in f.readline().rstrip(' \n').split()]
            edge_weight = [int(float(x)) for x in f.readline().rstrip(' \n').split()]
            node_bounds = [int(x) for x in f.readline().split()]
        node_bounds.append(len(edge_list))
        
        operations = dijkstra_save_operations(0, BinaryHeap(n), edge_list, edge_weight, node_bounds)
        path = os.path.join(DESTINATION, filename)
        with open(path, 'w') as f:
            for op_type in operations[0]:
                f.write(f'{op_type} ')
            f.write('\n')
            for op_args_1 in operations[1]:
                f.write(f'{op_args_1} ')
            f.write('\n')
            for op_args_2 in operations[2]:
                f.write(f'{op_args_2} ')
            f.write('\n')

In [27]:
"""dijkstra save operations worst case"""
import os
from heaps.binary_heap import BinaryHeap
GRAPH_NODES = [2_000, 4_000, 6_000, 8_000, 10_000, 20_000, 40_000, 60_000, 80_000, 100_000]

SOURCE_FOLDER = os.path.join(os.path.expanduser('~'), 'szakdolgozat', 'python', 'graphs', 'worst_case_graphs_10n_100')
DESTINATION = os.path.join(os.path.expanduser('~'), 'szakdolgozat', 'python', 'graph_operations', 'worst_10n_100',)

for n in GRAPH_NODES:

    edge_list = []
    edge_weight = []
    node_bounds = []

    # read the graph
    filename = f'{n}.txt'
    path = os.path.join(SOURCE_FOLDER, filename)
    with open(path, 'r') as f:
        edge_list = [int(x) for x in f.readline().rstrip(' \n').split()]
        edge_weight = [int(float(x)) for x in f.readline().rstrip(' \n').split()]
        node_bounds = [int(x) for x in f.readline().split()]
    node_bounds.append(len(edge_list))
    
    operations = dijkstra_save_operations(0, BinaryHeap(n), edge_list, edge_weight, node_bounds)
    path = os.path.join(DESTINATION, filename)
    with open(path, 'w') as f:
        for op_type in operations[0]:
            f.write(f'{op_type} ')
        f.write('\n')
        for op_args_1 in operations[1]:
            f.write(f'{op_args_1} ')
        f.write('\n')
        for op_args_2 in operations[2]:
            f.write(f'{op_args_2} ')
        f.write('\n')

In [6]:
""" Read test for dijkstra_operations """

GRAPH_NODES = [2_000, 4_000, 6_000, 8_000, 10_000, 20_000, 40_000, 60_000, 80_000, 100_000]
SAMPLE = 1
C = 1000
SOURCE_FOLDER = os.path.join(os.path.expanduser('~'), 'szakdolgozat', 'python', 'graph_operations', 'worst_10n_100')

n = 2000

# read the operations
filename = f'{n}.txt'
path = os.path.join(SOURCE_FOLDER, filename)
with open(path, 'r') as f:
    op_types = [x for x in f.readline().rstrip(' \n').split()]
    args_1 = [int(float(x)) for x in f.readline().rstrip(' \n').split()]
    args_2 = [int(float(x)) for x in f.readline().rstrip(' \n').split()]
#for i in range(len(op_types)):
#    print(op_types[i], args_1[i], args_2[i])
print(len(op_types) - 2*n)


18001


# worst case dijkstra

In [22]:
import os
import math

class Graph(object):
    """
        This calss is for weighted graphs, with nodes labeled from 0 to n, stores edge list
    """

    def __init__(self, n: int):
        """
            n is the maximum number of nodes.
        """
        self._edge_list = [[] for _ in range(n)]
        self._n = n
        self._m = 0

    @property
    def number_of_edges(self):
        return self._m

    @property
    def number_of_nodes(self):
        return self._n

    def __str__(self):
        return str(self._edge_list)

    def add_edge(self, edge):
        """
            edge is 3-tuple, or list: (node_1, node_2, weight)
        """
        node_1, node_2, weight = edge
        self._edge_list[node_1].append((node_2, weight))
        self._edge_list[node_2].append((node_1, weight))
        self._m += 1

    def add_edges(self, edges):
        """
            edges is a list of 3-tuples, or lists: (node_1, node_2, weight)
        """
        for edge in edges:
            self.add_edge(tuple(edge))

    def neighbors(self, node: int):
        return self._edge_list[node]

    def _set_edge_list(self, node: int, edgelist):
        """
            set edge list for a node, overwriting the current list.
            DO NOT CHANGE the other edge lists, therefore only use, if iterating through all nodes, and sets all edge_lists
        """
        self._edge_list[node] = edgelist.copy()

def worst_case_dijkstra_generator(n: int, m: int, c: int) -> tuple[list[int], list[int], list[int]]:
    """ Generates worst case graph according to the article
    input:
        n: number of nodes
        m: number of edges
        c: base edge-cost

    output:
        edge_list: list of edges from the 0-th node, ... n-th node
        edge_weight: list of weightst corresponding to the edges
        node_bounds: the i-th element is the first neighbor's place in edge_list of node i."""
    
    G = Graph(n)    # my Graph class
    afterstack = [] # the shortest path
    afterstack.extend([(x, x+1, c) for x in range(n-2, -1, -1)])    
    stack = []
    m1 = m - (n-1)      # Number ofthe additional edges
    i = 0
    j = i + 2
    while m1 > 0:
        if j < n:
            stack.append([i, j, 0])
            m1 -= 1
            j += 1
        else:
            i += 1
            j = i + 2
    if stack:
        edge = stack.pop()
        last_c = edge[2] = (n-edge[0])*c+1
        afterstack.append(tuple(edge))
        while stack:        # while its not empty
            edge = stack.pop()
            j = edge[1]
            if j == n-1:
                 last_c = edge[2] = last_c + c + 1
            else:
                last_c = edge[2] = last_c + 1
            afterstack.append(tuple(edge))

    afterstack.reverse()
    G.add_edges(afterstack)
    
    edge_list = []
    edge_weight= []
    node_bounds = []
    for u in range(G.number_of_nodes):
        node_bounds.append(len(edge_list))
        for v, c in G.neighbors(u):
            edge_list.append(v)
            edge_weight.append(c)
    return edge_list, edge_weight, node_bounds

In [26]:
# Generate worst case graphs

GRAPH_NODES = [2_000, 4_000, 6_000, 8_000, 10_000, 20_000, 40_000, 60_000, 80_000, 100_000]
#GRAPH_NODES = [200_000, 400_000, 600_000, 800_000, 1_000_000]
C = 100
DESTINATION_FOLDER = os.path.join(os.path.expanduser('~'), 'szakdolgozat', 'python', 'graphs', f'worst_case_graphs_10n_{C}')
for n in GRAPH_NODES:
    print(n, end="\t")
    m = 10*n
    edges, weights, bounds = worst_case_dijkstra_generator(n, m, C)
    file_name = f'{n}.txt'
    path = os.path.join(DESTINATION_FOLDER, file_name)
    with open(path, 'w') as f:
        for edge in edges:
            f.write(f'{edge} ')
        f.write('\n')
        for weight in weights:
            f.write(f'{weight} ')
        f.write('\n')
        for bound in bounds:
            f.write(f'{bound} ')
    print(max(weights))

2000	218001
4000	436001
6000	654001
8000	872001
10000	1090001
20000	2180001
40000	4360001
60000	6540001
80000	8720001
100000	10900001


In [6]:
"""give new weights for old graph"""
SAMPLE = 10
SOURCE_FOLDER = os.path.join(os.path.expanduser('~'), 'szakdolgozat', 'python', 'graphs', 'random_graphs_n32_2n')
DESTINATION_FOLDER = os.path.join(os.path.expanduser('~'), 'szakdolgozat', 'python', 'graphs', 'random_graphs_n32_2n_100')
GRAPH_NODES = [2_000, 4_000, 6_000, 8_000, 10_000, 20_000, 40_000, 60_000, 80_000, 100_000]

for n in GRAPH_NODES:
    C = 2*n
    print(n)
    for i in range(SAMPLE):
        filename = f'{n}_{i}.txt'
        path = os.path.join(SOURCE_FOLDER, filename)
        with open(path, 'r') as f:
            edge_list = [int(x) for x in f.readline().rstrip(' \n').split()]
            edge_weight = [int(float(x)) for x in f.readline().rstrip(' \n').split()]
            node_bounds = [int(x) for x in f.readline().split()]
        
        path = os.path.join(DESTINATION_FOLDER, filename)
        with open(path, 'w') as f:
            for edge in edge_list:
                f.write(f'{edge} ')
            f.write('\n')
            for weight in edge_weight:
                f.write(f'{weight*100} ')
            f.write('\n')
            for bound in node_bounds:
                f.write(f'{bound} ')

2000
4000
6000
8000
10000
20000
40000
60000
80000
100000
